In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
from dash import ctx
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "password"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({'animal_type':'Dog'}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(className='Title',
             children=[
                html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'150px',
                                                                                               'width' : '150px'}),
                #html.B(html.H1('Austin Animal Center Dashboard')),
                html.Div(className='subTitle',
                         children=[
                             html.B(html.H1('Austin Animal Control Dashboard')),
                             html.Br(),
                             html.Center(html.B(html.H4('LordSzeras, reporting for duty.')))],
                         style={
                             'whitespace' : 'space-between'
                         }
                         ),
                html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'130px',
                                                                                               'width' : '130px'})],
            style={
                'display' : 'flex',
                'whiteSpace' : 'pre-line',
                'justifyContent' : 'space-between',
                'alignItems' : 'center',
                'padding' : '20px'
            }),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    html.Div(className='sortByRescueButtons',
        style={'display' : 'flex'},
            children=[
                html.Button(id='water-button', n_clicks=0, children='Water'),
                html.Button(id='mountain-wild-button', n_clicks=0, children='Mountain/Wilderness'),
                html.Button(id='disaster-individual-button', n_clicks=0, children='Disaster/Individual'),
                html.Button(id='clear-button', n_clicks=0, children='Clear Filter')
            ]),
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        filter_action='native',
        editable=False,
        row_selectable='single',
        sort_action='native',
        sort_mode='multi',
        #column_selectable='single',
        page_size=6,
        selected_rows=[0]
        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# This callback will update the table to only display records
# of the animals corresponding to the button clicked
@app.callback(
    Output('datatable-id', 'data'),
    [Input('water-button', 'n_clicks'),
     Input('mountain-wild-button', 'n_clicks'),
     Input('disaster-individual-button', 'n_clicks'),
     Input('clear-button', 'n_clicks')
    ])
def on_click(button1, button2, button3, button4):
    df = pd.DataFrame.from_records(db.read({}))
    
    # use last button clicked
    if "water-button" == ctx.triggered_id:
        df = pd.DataFrame.from_records(db.read({'animal_type':'Dog',
                                                     'breed':{'$in':['Labrador Retriever Mix',
                                                                     'Chesapeake Bay Retriever',
                                                                     'Newfoundland']},
                                                    'sex_upon_outcome':'Intact Male',
                                                    'age_upon_outcome_in_weeks':{'$gte':26},
                                                    'age_upon_outcome_in_weeks':{'$lte':156}}))
    elif "mountain-wild-button" == ctx.triggered_id:
        df = pd.DataFrame.from_records(db.read({'animal_type':'Dog',
                                                     'breed':{'$in':['German Shepherd',
                                                                     'Alaskan Malamute',
                                                                     'Old English Sheepdog',
                                                                     'Siberian Husky',
                                                                     'Rottweiler']},
                                                    'sex_upon_outcome':'Intact Male',
                                                    'age_upon_outcome_in_weeks':{'$gte':26},
                                                    'age_upon_outcome_in_weeks':{'$lte':156}}))
    elif "disaster-individual-button" == ctx.triggered_id:
        df = pd.DataFrame.from_records(db.read({'animal_type':'Dog',
                                                     'breed':{'$in':['Doberman Pinscher',
                                                                     'German Shepherd',
                                                                     'Golden Retriever',
                                                                     'Bloodhound',
                                                                     'Rottweiler']},
                                                    'sex_upon_outcome':'Intact Male',
                                                    'age_upon_outcome_in_weeks':{'$gte':20},
                                                    'age_upon_outcome_in_weeks':{'$lte':300}}))
    elif "clear-button" == ctx.triggered_id:
        df = pd.DataFrame.from_records(db.read({'animal_type':'Dog'}))
    
    # drop _id field
    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):

    df = pd.DataFrame.from_records(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Breeds'),
            style={'width': '800px', 'height': '400px'}
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '800px', 'height': '400px'}, center=[30.75,-97.48], zoom=9, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:16653/
